In [89]:
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import pandas as pd
import warnings

In [90]:
# SQL User information
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
dst_dbname = "Walmart58"

In [91]:
# Read in the CSV file
walmart_db = pd.read_csv("walmart-sales-dataset-of-45stores.csv", delimiter = ",")
walmart_db.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [92]:
# Change the flags (0's or 1's) to true or false
walmart_db["Holiday_Flag"] = walmart_db["Holiday_Flag"].astype(bool)

In [93]:
walmart_db.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,False,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,True,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,False,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,False,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,False,46.50,2.625,211.350143,8.106


In [94]:
# Create a column that counts the row number, we'll use this later as our primary key
row_number=range(len(walmart_db))
walmart_db['sarah'] = row_number

In [95]:
walmart_db.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,sarah
0,1,05-02-2010,1643690.90,False,42.31,2.572,211.096358,8.106,0
1,1,12-02-2010,1641957.44,True,38.51,2.548,211.242170,8.106,1
2,1,19-02-2010,1611968.17,False,39.93,2.514,211.289143,8.106,2
3,1,26-02-2010,1409727.59,False,46.63,2.561,211.319643,8.106,3
4,1,05-03-2010,1554806.68,False,46.50,2.625,211.350143,8.106,4


In [96]:
# using alchemy to connect to mysql
def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY({pk_column});")
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    connection.close()
# Query to create Walmart database
exec_sql = f"CREATE DATABASE `{dst_dbname}`;"
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
sqlEngine.execute(exec_sql) #create db
sqlEngine.execute("USE Walmart58;") # select new db

In [97]:
# calling the function we just created
db_operation = "insert"
dataframe = walmart_db
table_name = "walmar"
primary_key = 'sarah'
set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [98]:
# Define a function to get a database in SQL
def get_dataframe(user_id, pwd, host_ip, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    return dframe

In [99]:
sql_walmart_db = "SELECT * FROM Walmart56.walmar;"
df_walmart = get_dataframe(user_id, pwd, host_name, dst_dbname,sql_walmart_db)
df_walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Fuel_Price,CPI,sarah
0,1,05-02-2010,1643690.90,0,2.572,211.096358,0
1,1,12-02-2010,1641957.44,1,2.548,211.242170,1
2,1,19-02-2010,1611968.17,0,2.514,211.289143,2
3,1,26-02-2010,1409727.59,0,2.561,211.319643,3
4,1,05-03-2010,1554806.68,0,2.625,211.350143,4


In [101]:
# SQL query to get the average gas price
average_query = "SELECT AVG(Fuel_Price) FROM Walmart56.walmar"
avg_df = get_dataframe(user_id, pwd, host_name, dst_dbname, average_query)
avg_df.head()

,AVG(Fuel_Price)
0,3.358607


In [102]:
# SQL query to get the average weekly sales
average_sales_query = "SELECT AVG(Weekly_Sales) FROM Walmart56.walmar"
avg_df2 = get_dataframe(user_id, pwd, host_name, dst_dbname, average_sales_query)
avg_df2.head()

,AVG(Weekly_Sales)
0,1.046965e+06


In [103]:
# SQL query to count number of weeks with a holiday
hday_query = "SELECT COUNT(Holiday_Flag)FROM Walmart56.walmar WHERE Holiday_Flag = 1"
hday_df = get_dataframe(user_id, pwd, host_name, dst_dbname, hday_query)
hday_df.head()

,COUNT(Holiday_Flag)
0,450
